In [51]:
import pandas as pd
import geopandas as gp
import os
import zipfile
import numpy as np
import scipy.cluster.vq
from scipy.cluster.vq import kmeans2
from sklearn.cluster import KMeans
import sklearn


## Importing & Cleaning NYC Zip Codes

In [ ]:
!curl -O https://raw.githubusercontent.com/fedhere/PUI2016_fb55/master/HW11_fb55/nyc-zip-code-tabulation-areas-polygons.geojson

In [ ]:
os.system("mv nyc-zip-code-tabulation-areas-polygons.geojson " + os.getenv("PUIDATA"))

In [5]:
zip_file = os.getenv('PUIDATA') + '/nyc-zip-code-tabulation-areas-polygons.geojson'
zip_file

'/home/cusp/lag552/PUIdata/nyc-zip-code-tabulation-areas-polygons.geojson'

In [6]:
# SOURCE: https://ocefpaf.github.io/python4oceanographers/blog/2015/03/30/geo_pandas/
zip_shape = gp.read_file(zip_file)
zip_shape.head()

,@id,BLDGpostalCode,CTY_FIPS,OBJECTID,PO_NAME,STATE,ST_FIPS,Shape_Area,Shape_Leng,borough,geometry,postalCode
0,http://nyc.pediacities.com/Resource/PostalCode...,0,081,1,Jackson Heights,NY,36,2.016328e+07,20624.692317,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,11372
1,http://nyc.pediacities.com/Resource/PostalCode...,0,081,2,Glen Oaks,NY,36,2.260653e+07,23002.816039,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,11004
2,http://nyc.pediacities.com/Resource/PostalCode...,0,081,3,New Hyde Park,NY,36,6.269333e+06,15749.161511,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,11040
3,http://nyc.pediacities.com/Resource/PostalCode...,0,081,4,Bellerose,NY,36,4.941836e+07,35932.810639,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,11426
4,http://nyc.pediacities.com/Resource/PostalCode...,0,081,5,Fresh Meadows,NY,36,6.938587e+07,38693.565676,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,11365


In [7]:
# SOURCE: http://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-of-certain-column-is-nan
zip_remnull = zip_shape[pd.notnull(zip_shape['postalCode'])]
zip_remnull.head()

,@id,BLDGpostalCode,CTY_FIPS,OBJECTID,PO_NAME,STATE,ST_FIPS,Shape_Area,Shape_Leng,borough,geometry,postalCode
0,http://nyc.pediacities.com/Resource/PostalCode...,0,081,1,Jackson Heights,NY,36,2.016328e+07,20624.692317,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,11372
1,http://nyc.pediacities.com/Resource/PostalCode...,0,081,2,Glen Oaks,NY,36,2.260653e+07,23002.816039,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,11004
2,http://nyc.pediacities.com/Resource/PostalCode...,0,081,3,New Hyde Park,NY,36,6.269333e+06,15749.161511,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,11040
3,http://nyc.pediacities.com/Resource/PostalCode...,0,081,4,Bellerose,NY,36,4.941836e+07,35932.810639,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,11426
4,http://nyc.pediacities.com/Resource/PostalCode...,0,081,5,Fresh Meadows,NY,36,6.938587e+07,38693.565676,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,11365


In [8]:
zip_clean = zip_remnull[['PO_NAME', 'STATE', 'borough', 'geometry', 'postalCode']]
zip_clean.head()

,PO_NAME,STATE,borough,geometry,postalCode
0,Jackson Heights,NY,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,11372
1,Glen Oaks,NY,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,11004
2,New Hyde Park,NY,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,11040
3,Bellerose,NY,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,11426
4,Fresh Meadows,NY,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,11365


In [9]:
len(zip_clean)

262

In [10]:
zip_clean.rename(columns = {"postalCode": "zip"}, inplace = True)
zip_clean.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,PO_NAME,STATE,borough,geometry,zip
0,Jackson Heights,NY,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,11372
1,Glen Oaks,NY,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,11004
2,New Hyde Park,NY,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,11040
3,Bellerose,NY,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,11426
4,Fresh Meadows,NY,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,11365


In [11]:
# From Santiago
zip_clean['zip'] = pd.to_numeric(zip_clean['zip'])
zip_clean['zip']

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


0      11372
1      11004
2      11040
3      11426
4      11365
5      11373
6      11001
7      11375
8      11427
9      11374
10     11366
11     11423
12     11428
13     11432
14     11379
15     11429
16     11435
17     11415
18     11418
19     11433
20     11451
21     11221
22     11421
23     11419
24     11434
25     11216
26     11416
27     11233
28     11436
29     11213
       ...  
232    10106
233    10158
234    10041
235    10120
236    10278
237    10155
238    10022
239    10043
240    10081
241    10096
242    10097
243    10196
244    10196
245    10275
246    10265
247    10045
248    10047
249    10047
250    10080
251    10203
252    10259
253    10260
254    10285
255    10286
256    11370
257    10065
258    10075
259    10069
260    10281
261    10282
Name: zip, dtype: int64

## Importing & Organizing Census Data

In [ ]:
!for ((y=93; y<=99; y+=1)); do wget ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done

!for ((y=0; y<=1; y+=1)); do wget ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=2; y<=9; y+=1)); do wget ftp://ftp.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=10; y<=15; y+=1)); do wget ftp://ftp.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done

In [12]:
years = ['94', '95', '96', '97', '98', '99', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14']
print(years)

['94', '95', '96', '97', '98', '99', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14']


In [13]:
for i in years:
    zf = zipfile.ZipFile('zbp'+ i +'totals.zip')
    est_data = pd.read_csv(zf.open('zbp'+ i +'totals.zip'.replace('.zip','.txt')))
    est_data.rename(columns = {"ZIP": "zip", "EST": "est"}, inplace = True)
    est_data = est_data[['zip', 'est']]
    est_data.rename(columns = {"est": "est" + i}, inplace = True)
    print("Year " + i)
    print(est_data.head())

Year 94
    zip  est94
0  1001    439
1  1002    450
2  1003     10
3  1004     47
4  1005     92
Year 95
    zip  est95
0   501      2
1  1000      4
2  1001    439
3  1002    467
4  1003     10
Year 96
    zip  est96
0   501      4
1  1000     11
2  1001    433
3  1002    487
4  1003     13
Year 97
    zip  est97
0   501      4
1  1000      8
2  1001    457
3  1002    500
4  1003     15
Year 98
    zip  est98
0   501      3
1  1001    444
2  1002    490
3  1003     14
4  1004     31
Year 99
    zip  est99
0   501      2
1  1001    469
2  1002    558
3  1003     13
4  1004     35
Year 00
    zip  est00
0   501      1
1  1001    493
2  1002    563
3  1003     13
4  1004     48
Year 01
    zip  est01
0   501      6
1  1001    490
2  1002    587
3  1003     13
4  1004     37
Year 02
    zip  est02
0   501     11
1  1001    490
2  1002    566
3  1003     15
4  1004     23
Year 03
    zip  est03
0   501      7
1  1001    505
2  1002    586
3  1003     16
4  1004     21
Year 04
    zip  est

## Merging Datasets

In [14]:
for i in years:
    zf = zipfile.ZipFile('zbp'+i+'totals.zip')
    temp = pd.read_csv(zf.open('zbp'+i+'totals.zip'.replace('.zip','.txt')))
    temp.columns = map(str.lower, temp.columns)
    temp = temp[['zip', 'est']]
    temp.rename(columns={'est' : 'est'+str(i)}, inplace=True)
    zip_clean = zip_clean.merge(temp, on = 'zip', how = 'inner')

In [15]:
zip_clean.shape

(243, 26)

In [16]:
zip_clean.columns

Index(['PO_NAME', 'STATE', 'borough', 'geometry', 'zip', 'est94', 'est95',
       'est96', 'est97', 'est98', 'est99', 'est00', 'est01', 'est02', 'est03',
       'est04', 'est05', 'est06', 'est07', 'est08', 'est09', 'est10', 'est11',
       'est12', 'est13', 'est14'],
      dtype='object')

In [17]:
zip_clean.head()

,PO_NAME,STATE,borough,geometry,zip,est94,est95,est96,est97,est98,...,est05,est06,est07,est08,est09,est10,est11,est12,est13,est14
0,Jackson Heights,NY,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,11372,1229,1236,1292,1309,1351,...,1539,1527,1545,1538,1572,1629,1647,1724,1759,1768
1,Glen Oaks,NY,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,11004,194,188,176,192,194,...,225,226,238,227,233,229,222,241,241,250
2,New Hyde Park,NY,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,11040,1422,1414,1335,1344,1344,...,1441,1454,1488,1503,1475,1494,1488,1510,1539,1534
3,Bellerose,NY,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,11426,326,219,209,198,203,...,226,226,228,226,237,248,256,266,258,260
4,Fresh Meadows,NY,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,11365,492,476,483,522,509,...,604,608,617,616,628,643,677,679,714,731


## Whitening Data

In [21]:
# From Santiago
# Other sources: 
# http://www.secnetix.de/olli/Python/lambda_functions.hawk
# http://pandas.pydata.org/pandas-docs/stable/indexing.html

est_mean = zip_clean.iloc[:,range(5,26)].mean(axis = 1)
est_std = zip_clean.iloc[:,range(5,26)].std(axis = 1)
zip_clean.iloc[:,range(5,26)] = zip_clean.iloc[:,range(5,26)].apply(lambda x: (x - est_mean) / est_std)
zip_clean.head()

,PO_NAME,STATE,borough,geometry,zip,est94,est95,est96,est97,est98,...,est05,est06,est07,est08,est09,est10,est11,est12,est13,est14
0,Jackson Heights,NY,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,11372,-1.553691,-1.553691,-1.553691,-1.553691,-0.534081,...,0.485528,0.485528,0.485528,0.485528,0.485528,0.485528,0.485528,1.505138,1.505138,1.505138
1,Glen Oaks,NY,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,11004,-0.485946,-1.413661,-1.413661,-1.413661,-0.485946,...,0.441769,0.441769,1.369485,0.441769,0.441769,0.441769,0.441769,1.369485,1.369485,1.369485
2,New Hyde Park,NY,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,11040,-0.341059,-0.341059,-1.236339,-1.236339,-1.236339,...,0.554221,0.554221,0.554221,1.449501,0.554221,0.554221,0.554221,1.449501,1.449501,1.449501
3,Bellerose,NY,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,11426,3.419555,-0.462102,-0.462102,-1.432516,-0.462102,...,-0.462102,-0.462102,-0.462102,-0.462102,0.508312,0.508312,0.508312,1.478727,0.508312,0.508312
4,Fresh Meadows,NY,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,11365,-1.478727,-1.478727,-1.478727,-0.508312,-1.478727,...,0.462102,0.462102,0.462102,0.462102,0.462102,0.462102,1.432516,1.432516,1.432516,1.432516


In [56]:
zip_clean.drop(zip_clean.columns[[0,2]], axis = 1, inplace = True)
zip_clean.head()

,geometry,est94,est95,est96,est97,est98,est99,est00,est01,est02,...,est05,est06,est07,est08,est09,est10,est11,est12,est13,est14
0,POLYGON ((-73.86942457284175 40.74915687096787...,-1.553691,-1.553691,-1.553691,-1.553691,-0.534081,-0.534081,-0.534081,-0.534081,-0.534081,...,0.485528,0.485528,0.485528,0.485528,0.485528,0.485528,0.485528,1.505138,1.505138,1.505138
1,POLYGON ((-73.71068374074007 40.75004039450917...,-0.485946,-1.413661,-1.413661,-1.413661,-0.485946,-1.413661,-1.413661,-0.485946,-0.485946,...,0.441769,0.441769,1.369485,0.441769,0.441769,0.441769,0.441769,1.369485,1.369485,1.369485
2,POLYGON ((-73.70098278625547 40.73889569923034...,-0.341059,-0.341059,-1.236339,-1.236339,-1.236339,-1.236339,-1.236339,-1.236339,-0.341059,...,0.554221,0.554221,0.554221,1.449501,0.554221,0.554221,0.554221,1.449501,1.449501,1.449501
3,POLYGON ((-73.72270447144122 40.75373371438336...,3.419555,-0.462102,-0.462102,-1.432516,-0.462102,-0.462102,-0.462102,-0.462102,-0.462102,...,-0.462102,-0.462102,-0.462102,-0.462102,0.508312,0.508312,0.508312,1.478727,0.508312,0.508312
4,POLYGON ((-73.81088634744755 40.72717187575918...,-1.478727,-1.478727,-1.478727,-0.508312,-1.478727,-0.508312,-0.508312,-0.508312,-0.508312,...,0.462102,0.462102,0.462102,0.462102,0.462102,0.462102,1.432516,1.432516,1.432516,1.432516


## Clustering Data

2. cluster the NUMBER OF ESTABLISHMENTS time series with K-means in **a few** clusters (as discussed there is no real good, sound way to decide what a good number is here. try a few options, keeping in mind a few is more than a couple, but i recommand you stay within the single digit numbers)
3. plot the cluster centers (if you used K means those are the means of the clusters). you can plot for example the cluster centers overlayed on each time series (using the alpha channel to control the opacity in the plot may be helpful here).
4. Use another clustering algorithm (of your choice)
5. overlay your data on a NYC map: you can use shapefiles for the zip codes and different colors for different clusters
6. Compare the results of the 2 algorithms
7. attempt an interpretation. this is dangerous ground: clustering is an exploratory tool so you do not want to jump to conclusions because you see some clusters! but seeing structure in your data can inform your next moves as an investigator.